In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
%matplotlib inline

In [4]:
dataset = os.path.join(parentdir,'ml-latest-small/ratings.1.csv')    
ratings = pd.read_csv(dataset, sep=',',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)


#dataset = os.path.join(parentdir,'movielens100k/u.data')     
#ratings = pd.read_csv(dataset, sep='\s+',
#                      names=['user', 'item', 'rating', 'timestamp'], header=0)

items = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
                      names=['item','title','genres'], header=0)

In [5]:
items_dummy = pd.concat([items, items['genres'].str.get_dummies(sep='|')], axis=1)
items_dummy.head()

,item,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
items_n = ratings.item.nunique()
items_n

385

In [7]:
algo_uu = uknn.UserUser(20)
algo_ii = iknn.ItemItem(20)
algo_als = als.BiasedMF(50)
algo_funkSVD = funksvd.FunkSVD(50)


In [8]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    
    # now we run the recommender
    recs = batch.recommend(fittable, users, items_n)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [9]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    all_recs.append(eval('FunkSVD', algo_funkSVD, train, test))
    

In [10]:
test_data = pd.concat(test_data, ignore_index=True)
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs_joined = all_recs.join(items_dummy.set_index('item'), on='item')
all_recs_joined.head()

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2081,4.602925,2,1,ItemItem,"Little Mermaid, The (1989)",Animation|Children|Comedy|Musical|Romance,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,296,4.411197,2,2,ItemItem,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1721,4.226864,2,3,ItemItem,Titanic (1997),Drama|Romance,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,141,4.024384,2,4,ItemItem,"Birdcage, The (1996)",Comedy,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1961,4.024384,2,5,ItemItem,Rain Man (1988),Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#test = all_recs_joined.loc[all_recs_joined["user"]==1]
#test = test.loc[test["Algorithm"] == "ALS"]
#test.reset_index()
#test.iloc[90:100,:]

In [12]:
#providers = list(all_recs_joined.iloc[:,8:])
#items_n = all_recs_joined.item.nunique()
#proItems_n =  all_recs_joined.loc[ all_recs_joined['Drama'] == 1].item.nunique()
#proItems_n_horror =  all_recs_joined.loc[ all_recs_joined['Horror'] == 1].item.nunique()

In [13]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results = rla.compute(all_recs_joined, test_data, "Drama", providers, items_n, proItems_n)
#results.head()

In [14]:
#results.groupby('Algorithm').mean()

In [15]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results_horror = rla.compute(all_recs_joined, test_data, "Horror", providers, items_n, proItems_n_horror)
#results_horror.groupby('Algorithm').mean()

In [16]:
#allResults = []
#allResults.append(results.groupby('Algorithm').mean())
#allResults.append(results_horror.groupby('Algorithm').mean())

In [17]:
#pd.concat(allResults, keys=['Drama', 'Horror'])

In [18]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [19]:
#testproviders = ["Drama", "Action"]
testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

Action


C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


done Action
Adventure
done Adventure
Animation
done Animation
Children
done Children
Comedy
done Comedy
Crime
done Crime
Documentary
done Documentary
Drama
done Drama
Fantasy
done Fantasy
Film-Noir
done Film-Noir
Horror
done Horror
IMAX
done IMAX
Musical
done Musical
Mystery
done Mystery
Romance
done Romance
Sci-Fi
done Sci-Fi
Thriller
done Thriller
War
done War
Western
done Western


rND       rKL       rRD      APCR   nd_APCR  \
            Algorithm                                                     
Action      ALS        0.313273  0.028355  1.097398  0.989474  0.913124   
            FunkSVD    0.317979  0.048190  1.671447  0.989474  0.917010   
            ItemItem   0.355199  0.120878  0.247797  0.736842  0.638678   
            UserUser   0.313035  0.058905  1.456259  0.947368  0.890670   
Adventure   ALS        0.248034  0.026180  0.957688  0.989474  0.913124   
            FunkSVD    0.309751  0.055286  1.840018  0.989474  0.917010   
            ItemItem   0.266837  0.070701  0.386422  0.736842  0.638678   
            UserUser   0.272144  0.056627  1.262511  0.947368  0.890670   
Animation   ALS        0.215398  0.032362  0.469960  0.989474  0.913124   
            FunkSVD    0.202706  0.026181  0.183786  0.989474  0.917010   
            ItemItem   0.282503  0.286169  0.180604  0.736842  0.638678   
            UserUser   0.226371  0.038889  0.120195  0.947368  0.890670   
Children    ALS        0.429336  0.070167  0.734907  0.989474  0.913124   
            FunkSVD    0.371526  0.053309  0.420170  0.989474  0.917010   
            ItemItem   0.331227  0.353248  0.228129  0.736842  0.638678   
            UserUser   0.375684  0.070292  0.319276  0.947368  0.890670   
Comedy      ALS        0.571578  0.084506  5.390105  0.989474  0.913124   
            FunkSVD    0.517054  0.061718  5.474443  0.989474  0.917010   
            ItemItem   0.190809  0.040640  0.383854  0.736842  0.638678   
            UserUser   0.732947  0.150563  5.641272  0.947368  0.890670   
Crime       ALS        0.595354  0.120514  0.327757  0.989474  0.913124   
            FunkSVD    0.365905  0.042886  0.494776  0.989474  0.917010   
            ItemItem   0.260498  0.070189  0.027963  0.736842  0.638678   
            UserUser   0.239810  0.043576  0.602102  0.947368  0.890670   
Documentary ALS        0.044129  0.010336  0.090943  0.989474  0.913124   
            FunkSVD    0.041013  0.009780  0.139001  0.989474  0.917010   
            ItemItem   0.000000  0.000000  0.000000  0.736842  0.638678   
            UserUser   0.050683  0.017920  0.111650  0.947368  0.890670   
Drama       ALS        0.934787  0.153799  4.642282  0.989474  0.913124   
            FunkSVD    0.599847  0.070769  1.736634  0.989474  0.917010   
...                         ...       ...       ...       ...       ...   
IMAX        ItemItem   0.157618  0.097639  0.156726  0.736842  0.638678   
            UserUser   0.145446  0.073505  0.236348  0.947368  0.890670   
Musical     ALS        0.314029  0.060061  0.646273  0.989474  0.913124   
            FunkSVD    0.516961  0.114170  0.017644  0.989474  0.917010   
            ItemItem   0.240068  0.135712  0.061224  0.736842  0.638678   
            UserUser   0.443918  0.121189  0.002854  0.947368  0.890670   
Mystery     ALS        0.254397  0.063254  0.189728  0.989474  0.913124   
            FunkSVD    0.148018  0.038900  0.498099  0.989474  0.917010   
            ItemItem   0.053174  0.038297  0.036700  0.736842  0.638678   
            UserUser   0.162388  0.049471  0.473001  0.947368  0.890670   
Romance     ALS        0.402146  0.058914  1.838061  0.989474  0.913124   
            FunkSVD    0.528429  0.111598  0.422190  0.989474  0.917010   
            ItemItem   0.124426  0.023833  0.041146  0.736842  0.638678   
            UserUser   0.397913  0.071640  0.250427  0.947368  0.890670   
Sci-Fi      ALS        0.327883  0.056569  0.471322  0.989474  0.913124   
            FunkSVD    0.243675  0.042518  1.034764  0.989474  0.917010   
            ItemItem   0.268577  0.113707  0.262391  0.736842  0.638678   
            UserUser   0.232900  0.048917  0.918683  0.947368  0.890670   
Thriller    ALS        0.411756  0.055656  1.414684  0.989474  0.913124   
            FunkSVD    0.502212  0.077399  3.084543  0.989474  0.917010   
            ItemItem   0.279542  0.117559  0.032665  

In [23]:
output = pd.concat(allResults, keys=testproviders)
output

rND       rKL       rRD      APCR   nd_APCR  \
            Algorithm                                                     
Action      ALS        0.313273  0.028355  1.097398  0.989474  0.913124   
            FunkSVD    0.317979  0.048190  1.671447  0.989474  0.917010   
            ItemItem   0.355199  0.120878  0.247797  0.736842  0.638678   
            UserUser   0.313035  0.058905  1.456259  0.947368  0.890670   
Adventure   ALS        0.248034  0.026180  0.957688  0.989474  0.913124   
            FunkSVD    0.309751  0.055286  1.840018  0.989474  0.917010   
            ItemItem   0.266837  0.070701  0.386422  0.736842  0.638678   
            UserUser   0.272144  0.056627  1.262511  0.947368  0.890670   
Animation   ALS        0.215398  0.032362  0.469960  0.989474  0.913124   
            FunkSVD    0.202706  0.026181  0.183786  0.989474  0.917010   
            ItemItem   0.282503  0.286169  0.180604  0.736842  0.638678   
            UserUser   0.226371  0.038889  0.120195  0.947368  0.890670   
Children    ALS        0.429336  0.070167  0.734907  0.989474  0.913124   
            FunkSVD    0.371526  0.053309  0.420170  0.989474  0.917010   
            ItemItem   0.331227  0.353248  0.228129  0.736842  0.638678   
            UserUser   0.375684  0.070292  0.319276  0.947368  0.890670   
Comedy      ALS        0.571578  0.084506  5.390105  0.989474  0.913124   
            FunkSVD    0.517054  0.061718  5.474443  0.989474  0.917010   
            ItemItem   0.190809  0.040640  0.383854  0.736842  0.638678   
            UserUser   0.732947  0.150563  5.641272  0.947368  0.890670   
Crime       ALS        0.595354  0.120514  0.327757  0.989474  0.913124   
            FunkSVD    0.365905  0.042886  0.494776  0.989474  0.917010   
            ItemItem   0.260498  0.070189  0.027963  0.736842  0.638678   
            UserUser   0.239810  0.043576  0.602102  0.947368  0.890670   
Documentary ALS        0.044129  0.010336  0.090943  0.989474  0.913124   
            FunkSVD    0.041013  0.009780  0.139001  0.989474  0.917010   
            ItemItem   0.000000  0.000000  0.000000  0.736842  0.638678   
            UserUser   0.050683  0.017920  0.111650  0.947368  0.890670   
Drama       ALS        0.934787  0.153799  4.642282  0.989474  0.913124   
            FunkSVD    0.599847  0.070769  1.736634  0.989474  0.917010   
...                         ...       ...       ...       ...       ...   
IMAX        ItemItem   0.157618  0.097639  0.156726  0.736842  0.638678   
            UserUser   0.145446  0.073505  0.236348  0.947368  0.890670   
Musical     ALS        0.314029  0.060061  0.646273  0.989474  0.913124   
            FunkSVD    0.516961  0.114170  0.017644  0.989474  0.917010   
            ItemItem   0.240068  0.135712  0.061224  0.736842  0.638678   
            UserUser   0.443918  0.121189  0.002854  0.947368  0.890670   
Mystery     ALS        0.254397  0.063254  0.189728  0.989474  0.913124   
            FunkSVD    0.148018  0.038900  0.498099  0.989474  0.917010   
            ItemItem   0.053174  0.038297  0.036700  0.736842  0.638678   
            UserUser   0.162388  0.049471  0.473001  0.947368  0.890670   
Romance     ALS        0.402146  0.058914  1.838061  0.989474  0.913124   
            FunkSVD    0.528429  0.111598  0.422190  0.989474  0.917010   
            ItemItem   0.124426  0.023833  0.041146  0.736842  0.638678   
            UserUser   0.397913  0.071640  0.250427  0.947368  0.890670   
Sci-Fi      ALS        0.327883  0.056569  0.471322  0.989474  0.913124   
            FunkSVD    0.243675  0.042518  1.034764  0.989474  0.917010   
            ItemItem   0.268577  0.113707  0.262391  0.736842  0.638678   
            UserUser   0.232900  0.048917  0.918683  0.947368  0.890670   
Thriller    ALS        0.411756  0.055656  1.414684  0.989474  0.913124   
            FunkSVD    0.502212  0.077399  3.084543  0.989474  0.917010   
            ItemItem   0.279542  0.117559  0.032665  